<a class="anchor" id="0"></a>
# [OpenVaccine: COVID-19 mRNA Vaccine Degradation Prediction](https://www.kaggle.com/c/stanford-covid-vaccine)

### I use the notebook [GRU & LSTM mix with custom loss](https://www.kaggle.com/gandagorn/gru-lstm-mix-with-custom-loss) from [Daniel Gärber](https://www.kaggle.com/gandagorn), which used the notebook [mVAN : COVID mRNA Vaccine Analysis Notebook [.268]](https://www.kaggle.com/aestheteaman01/mvan-covid-mrna-vaccine-analysis-notebook-268) from [Aman Kumar](https://www.kaggle.com/aestheteaman01), as a basis and will try to tune its various parameters

# Acknowledgements

* notebook [GRU & LSTM mix with custom loss](https://www.kaggle.com/gandagorn/gru-lstm-mix-with-custom-loss) as a basis
* notebook [mVAN : COVID mRNA Vaccine Analysis Notebook [.268]](https://www.kaggle.com/aestheteaman01/mvan-covid-mrna-vaccine-analysis-notebook-268) as basis for the first notebook 
* notebook [Higher LB score by tuning mloss - upgrade & visual](https://www.kaggle.com/vbmokin/higher-lb-score-by-tuning-mloss-upgrade-visual) as a sample of tunning 

### My upgrade: the structure of the model and its training

<a class="anchor" id="0.1"></a>
## Table of Contents

1. [Import libraries](#1)
1. [My upgrade](#2)
    -  [Commit now](#2.1)
    -  [Previous commits](#2.2)
    -  [Parameters and LB score visualization](#2.3)
1. [Download data, auxiliary functions and model tuning](#3)
1. [Ensembling solutions and submission](#4)

## 1. Import libraries <a class="anchor" id="1"></a>

[Back to Table of Contents](#0.1)

In [ ]:
import os 
import sys
import json
import math
import random
import numpy as np
import pandas as pd
import gc
from tqdm import tqdm

import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split, KFold,  StratifiedKFold

import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L

import warnings
warnings.filterwarnings("ignore")

In [ ]:
seed = 42

## 2. My upgrade <a class="anchor" id="2"></a>

[Back to Table of Contents](#0.1)

The structure from https://www.kaggle.com/vbmokin/higher-lb-score-by-tuning-mloss-upgrade-visual

## 2.1. Commit now <a class="anchor" id="2.1"></a>

[Back to Table of Contents](#0.1)

In [ ]:
dropout_model = 0.36
hidden_dim_first = 128
hidden_dim_second = 248
hidden_dim_third = 212

## 2.2 Previous commits <a class="anchor" id="2.2"></a>

[Back to Table of Contents](#0.1)

In [ ]:
commits_df = pd.DataFrame(columns = ['commit_num', 'dropout_model', 'hidden_dim_first', 'hidden_dim_second', 'hidden_dim_third', 'LB_score'])

### Commit 0 (parameters from https://www.kaggle.com/gandagorn/gru-lstm-mix-with-custom-loss, commit 8)

* dropout_model = 0.4
* hidden_dim_first = 128
* hidden_dim_second = 128
* hidden_dim_third = 128

LB = 0.25883

In [ ]:
n=0
commits_df.loc[n,'commit_num'] = 0
commits_df.loc[n,'dropout_model'] = 0.4
commits_df.loc[n,'hidden_dim_first'] = 128
commits_df.loc[n,'hidden_dim_second'] = 128
commits_df.loc[n,'hidden_dim_third'] = 128
commits_df.loc[n,'LB_score'] = 0.25883

### Commit 3

* dropout_model = 0.36
* hidden_dim_first = 128
* hidden_dim_second = 256
* hidden_dim_third = 128

LB = 0.25855

In [ ]:
n=1
commits_df.loc[n,'commit_num'] = 3
commits_df.loc[n,'dropout_model'] = 0.36
commits_df.loc[n,'hidden_dim_first'] = 128
commits_df.loc[n,'hidden_dim_second'] = 256
commits_df.loc[n,'hidden_dim_third'] = 128
commits_df.loc[n,'LB_score'] = 0.25855

### Commit 4

* dropout_model = 0.36
* hidden_dim_first = 256
* hidden_dim_second = 256
* hidden_dim_third = 128

LB = 0.25887

In [ ]:
n=2
commits_df.loc[n,'commit_num'] = 4
commits_df.loc[n,'dropout_model'] = 0.36
commits_df.loc[n,'hidden_dim_first'] = 256
commits_df.loc[n,'hidden_dim_second'] = 256
commits_df.loc[n,'hidden_dim_third'] = 128
commits_df.loc[n,'LB_score'] = 0.25887

### Commit 5

* dropout_model = 0.36
* hidden_dim_first = 128
* hidden_dim_second = 384
* hidden_dim_third = 128

LB = 0.25880

In [ ]:
n=3
commits_df.loc[n,'commit_num'] = 5
commits_df.loc[n,'dropout_model'] = 0.36
commits_df.loc[n,'hidden_dim_first'] = 128
commits_df.loc[n,'hidden_dim_second'] = 384
commits_df.loc[n,'hidden_dim_third'] = 128
commits_df.loc[n,'LB_score'] = 0.25880

### Commit 6

* dropout_model = 0.36
* hidden_dim_first = 384
* hidden_dim_second = 128
* hidden_dim_third = 128

LB = 0.25989

In [ ]:
n=4
commits_df.loc[n,'commit_num'] = 6
commits_df.loc[n,'dropout_model'] = 0.36
commits_df.loc[n,'hidden_dim_first'] = 384
commits_df.loc[n,'hidden_dim_second'] = 128
commits_df.loc[n,'hidden_dim_third'] = 128
commits_df.loc[n,'LB_score'] = 0.25989

### Commit 7

* dropout_model = 0.36
* hidden_dim_first = 128
* hidden_dim_second = 192
* hidden_dim_third = 128

LB = 0.25877

In [ ]:
n=5
commits_df.loc[n,'commit_num'] = 7
commits_df.loc[n,'dropout_model'] = 0.36
commits_df.loc[n,'hidden_dim_first'] = 128
commits_df.loc[n,'hidden_dim_second'] = 192
commits_df.loc[n,'hidden_dim_third'] = 128
commits_df.loc[n,'LB_score'] = 0.25877

### Commit 8

* dropout_model = 0.36
* hidden_dim_first = 128
* hidden_dim_second = 224
* hidden_dim_third = 128

LB = 0.25868

In [ ]:
n=6
commits_df.loc[n,'commit_num'] = 8
commits_df.loc[n,'dropout_model'] = 0.36
commits_df.loc[n,'hidden_dim_first'] = 128
commits_df.loc[n,'hidden_dim_second'] = 224
commits_df.loc[n,'hidden_dim_third'] = 128
commits_df.loc[n,'LB_score'] = 0.25868

### Commit 9

* dropout_model = 0.36
* hidden_dim_first = 128
* hidden_dim_second = 288
* hidden_dim_third = 128

LB = 0.25881

In [ ]:
n=7
commits_df.loc[n,'commit_num'] = 9
commits_df.loc[n,'dropout_model'] = 0.36
commits_df.loc[n,'hidden_dim_first'] = 128
commits_df.loc[n,'hidden_dim_second'] = 288
commits_df.loc[n,'hidden_dim_third'] = 128
commits_df.loc[n,'LB_score'] = 0.25881

### Commit 10

* dropout_model = 0.36
* hidden_dim_first = 128
* hidden_dim_second = 248
* hidden_dim_third = 128

LB = 0.25850

In [ ]:
n=8
commits_df.loc[n,'commit_num'] = 10
commits_df.loc[n,'dropout_model'] = 0.36
commits_df.loc[n,'hidden_dim_first'] = 128
commits_df.loc[n,'hidden_dim_second'] = 248
commits_df.loc[n,'hidden_dim_third'] = 128
commits_df.loc[n,'LB_score'] = 0.25850

### Commit 11

* dropout_model = 0.36
* hidden_dim_first = 128
* hidden_dim_second = 240
* hidden_dim_third = 128

LB = 0.25868

In [ ]:
n=9
commits_df.loc[n,'commit_num'] = 11
commits_df.loc[n,'dropout_model'] = 0.36
commits_df.loc[n,'hidden_dim_first'] = 128
commits_df.loc[n,'hidden_dim_second'] = 240
commits_df.loc[n,'hidden_dim_third'] = 128
commits_df.loc[n,'LB_score'] = 0.25868

### Commit 12

* dropout_model = 0.35
* hidden_dim_first = 128
* hidden_dim_second = 248
* hidden_dim_third = 128

LB = 0.25884

In [ ]:
n=10
commits_df.loc[n,'commit_num'] = 12
commits_df.loc[n,'dropout_model'] = 0.35
commits_df.loc[n,'hidden_dim_first'] = 128
commits_df.loc[n,'hidden_dim_second'] = 248
commits_df.loc[n,'hidden_dim_third'] = 128
commits_df.loc[n,'LB_score'] = 0.25884

### Commit 13

* dropout_model = 0.3
* hidden_dim_first = 128
* hidden_dim_second = 248
* hidden_dim_third = 128

LB = 0.25879

In [ ]:
n=11
commits_df.loc[n,'commit_num'] = 13
commits_df.loc[n,'dropout_model'] = 0.3
commits_df.loc[n,'hidden_dim_first'] = 128
commits_df.loc[n,'hidden_dim_second'] = 248
commits_df.loc[n,'hidden_dim_third'] = 128
commits_df.loc[n,'LB_score'] = 0.25879

### Commit 15

* dropout_model = 0.45
* hidden_dim_first = 128
* hidden_dim_second = 248
* hidden_dim_third = 128

LB = 0.25920

In [ ]:
n=12
commits_df.loc[n,'commit_num'] = 15
commits_df.loc[n,'dropout_model'] = 0.45
commits_df.loc[n,'hidden_dim_first'] = 128
commits_df.loc[n,'hidden_dim_second'] = 248
commits_df.loc[n,'hidden_dim_third'] = 128
commits_df.loc[n,'LB_score'] = 0.25920

### Commit 17

* dropout_model = 0.37
* hidden_dim_first = 128
* hidden_dim_second = 248
* hidden_dim_third = 128

LB = 0.25885

In [ ]:
n=13
commits_df.loc[n,'commit_num'] = 17
commits_df.loc[n,'dropout_model'] = 0.37
commits_df.loc[n,'hidden_dim_first'] = 128
commits_df.loc[n,'hidden_dim_second'] = 248
commits_df.loc[n,'hidden_dim_third'] = 128
commits_df.loc[n,'LB_score'] = 0.25885

### Commit 18

* dropout_model = 0.36
* hidden_dim_first = 128
* hidden_dim_second = 248
* hidden_dim_third = 248

LB = 0.25841

In [ ]:
n=14
commits_df.loc[n,'commit_num'] = 18
commits_df.loc[n,'dropout_model'] = 0.36
commits_df.loc[n,'hidden_dim_first'] = 128
commits_df.loc[n,'hidden_dim_second'] = 248
commits_df.loc[n,'hidden_dim_third'] = 248
commits_df.loc[n,'LB_score'] = 0.25841

### Commit 19

* dropout_model = 0.36
* hidden_dim_first = 128
* hidden_dim_second = 248
* hidden_dim_third = 248

LB = 0.25956

In [ ]:
n=15
commits_df.loc[n,'commit_num'] = 19
commits_df.loc[n,'dropout_model'] = 0.36
commits_df.loc[n,'hidden_dim_first'] = 128
commits_df.loc[n,'hidden_dim_second'] = 248
commits_df.loc[n,'hidden_dim_third'] = 384
commits_df.loc[n,'LB_score'] = 0.25956

### Commit 20

* dropout_model = 0.36
* hidden_dim_first = 128
* hidden_dim_second = 128
* hidden_dim_third = 248

LB = 0.25891

In [ ]:
n=16
commits_df.loc[n,'commit_num'] = 20
commits_df.loc[n,'dropout_model'] = 0.36
commits_df.loc[n,'hidden_dim_first'] = 128
commits_df.loc[n,'hidden_dim_second'] = 128
commits_df.loc[n,'hidden_dim_third'] = 248
commits_df.loc[n,'LB_score'] = 0.25891

### Commit 21

* dropout_model = 0.36
* hidden_dim_first = 128
* hidden_dim_second = 248
* hidden_dim_third = 240

LB = 0.25831

In [ ]:
n=17
commits_df.loc[n,'commit_num'] = 21
commits_df.loc[n,'dropout_model'] = 0.36
commits_df.loc[n,'hidden_dim_first'] = 128
commits_df.loc[n,'hidden_dim_second'] = 248
commits_df.loc[n,'hidden_dim_third'] = 240
commits_df.loc[n,'LB_score'] = 0.25831

### Commit 22

* dropout_model = 0.36
* hidden_dim_first = 128
* hidden_dim_second = 248
* hidden_dim_third = 216

LB = 0.25827

In [ ]:
n=18
commits_df.loc[n,'commit_num'] = 22
commits_df.loc[n,'dropout_model'] = 0.36
commits_df.loc[n,'hidden_dim_first'] = 128
commits_df.loc[n,'hidden_dim_second'] = 248
commits_df.loc[n,'hidden_dim_third'] = 216
commits_df.loc[n,'LB_score'] = 0.25827

### Commit 23

* dropout_model = 0.36
* hidden_dim_first = 128
* hidden_dim_second = 248
* hidden_dim_third = 184

LB = 0.25898

In [ ]:
n=19
commits_df.loc[n,'commit_num'] = 23
commits_df.loc[n,'dropout_model'] = 0.36
commits_df.loc[n,'hidden_dim_first'] = 128
commits_df.loc[n,'hidden_dim_second'] = 248
commits_df.loc[n,'hidden_dim_third'] = 184
commits_df.loc[n,'LB_score'] = 0.25898

### Commit 24

* dropout_model = 0.36
* hidden_dim_first = 128
* hidden_dim_second = 248
* hidden_dim_third = 208

LB = 0.25868

In [ ]:
n=20
commits_df.loc[n,'commit_num'] = 24
commits_df.loc[n,'dropout_model'] = 0.36
commits_df.loc[n,'hidden_dim_first'] = 128
commits_df.loc[n,'hidden_dim_second'] = 248
commits_df.loc[n,'hidden_dim_third'] = 208
commits_df.loc[n,'LB_score'] = 0.25868

### Commit 25

* dropout_model = 0.36
* hidden_dim_first = 128
* hidden_dim_second = 248
* hidden_dim_third = 212

LB = 0.25823

In [ ]:
n=21
commits_df.loc[n,'commit_num'] = 25
commits_df.loc[n,'dropout_model'] = 0.36
commits_df.loc[n,'hidden_dim_first'] = 128
commits_df.loc[n,'hidden_dim_second'] = 248
commits_df.loc[n,'hidden_dim_third'] = 212
commits_df.loc[n,'LB_score'] = 0.25823

### Commit 26

* dropout_model = 0.36
* hidden_dim_first = 128
* hidden_dim_second = 248
* hidden_dim_third = 210

LB = 0.25844

In [ ]:
n=22
commits_df.loc[n,'commit_num'] = 26
commits_df.loc[n,'dropout_model'] = 0.36
commits_df.loc[n,'hidden_dim_first'] = 128
commits_df.loc[n,'hidden_dim_second'] = 248
commits_df.loc[n,'hidden_dim_third'] = 210
commits_df.loc[n,'LB_score'] = 0.25844

### Commit 27

* dropout_model = 0.36
* hidden_dim_first = 64
* hidden_dim_second = 124
* hidden_dim_third = 108

LB = 0.25963

In [ ]:
n=23
commits_df.loc[n,'commit_num'] = 27
commits_df.loc[n,'dropout_model'] = 0.36
commits_df.loc[n,'hidden_dim_first'] = 64
commits_df.loc[n,'hidden_dim_second'] = 124
commits_df.loc[n,'hidden_dim_third'] = 108
commits_df.loc[n,'LB_score'] = 0.25963

### Commit 28

* dropout_model = 0.36
* hidden_dim_first = 128
* hidden_dim_second = 248
* hidden_dim_third = 108

LB = 0.25860

In [ ]:
n=24
commits_df.loc[n,'commit_num'] = 28
commits_df.loc[n,'dropout_model'] = 0.36
commits_df.loc[n,'hidden_dim_first'] = 128
commits_df.loc[n,'hidden_dim_second'] = 248
commits_df.loc[n,'hidden_dim_third'] = 108
commits_df.loc[n,'LB_score'] = 0.25860

### Commit 29

* dropout_model = 0.36
* hidden_dim_first = 128
* hidden_dim_second = 244
* hidden_dim_third = 128

LB = 0.25846

In [ ]:
n=25
commits_df.loc[n,'commit_num'] = 29
commits_df.loc[n,'dropout_model'] = 0.36
commits_df.loc[n,'hidden_dim_first'] = 128
commits_df.loc[n,'hidden_dim_second'] = 244
commits_df.loc[n,'hidden_dim_third'] = 128
commits_df.loc[n,'LB_score'] = 0.25846

## 2.3 Parameters and LB score visualization <a class="anchor" id="2.3"></a>

[Back to Table of Contents](#0.1)

In [ ]:
# Find and mark maximum value of LB score
commits_df['LB_score'] = pd.to_numeric(commits_df['LB_score'])
commits_df['best'] = 0
commits_df.loc[commits_df['LB_score'].idxmin(), 'best'] = 1

In [ ]:
commits_df.sort_values(by=['LB_score'])

In [ ]:
# Interactive plot with results of parameters tuning
fig = px.scatter_3d(commits_df, x='hidden_dim_first', y='hidden_dim_second', z='LB_score', color = 'best', 
                    symbol = 'dropout_model',
                    title='hidden_dim_1st & 2nd and LB score visualization of COVID-19 mRNA VDP solutions')
fig.update(layout=dict(title=dict(x=0.07)))

In [ ]:
# Interactive plot with results of parameters tuning
fig = px.scatter_3d(commits_df, x='hidden_dim_second', y='dropout_model', z='LB_score', color = 'best', 
                    symbol = 'hidden_dim_first',
                    title='hidden_dim_2nd & dropout and LB score visualization of COVID-19 mRNA VDP solutions')
fig.update(layout=dict(title=dict(x=0.07)))

In [ ]:
# Interactive plot with results of parameters tuning
fig = px.scatter_3d(commits_df, x='hidden_dim_second', y='hidden_dim_third', z='LB_score', color = 'best', 
                    symbol = 'hidden_dim_first',
                    title='hidden_dim_2nd & 3nd and LB score visualization of COVID-19 mRNA VDP solutions')
fig.update(layout=dict(title=dict(x=0.07)))

## 3. Download data, auxiliary functions and model tuning <a class="anchor" id="3"></a>

[Back to Table of Contents](#0.1)

From notebook https://www.kaggle.com/gandagorn/gru-lstm-mix-with-custom-loss

My upgrade: structure of model

In [ ]:
# Download datasets
train = pd.read_json('/kaggle/input/stanford-covid-vaccine/train.json', lines=True)
test = pd.read_json('/kaggle/input/stanford-covid-vaccine/test.json', lines=True)
sample_sub = pd.read_csv("/kaggle/input/stanford-covid-vaccine/sample_submission.csv")

In [ ]:
# Target columns 
target_cols = ['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']

In [ ]:
token2int = {x:i for i, x in enumerate('().ACGUBEHIMSX')}

def get_pair_index_structure(structure):
    structure = np.array([struc for struc in structure], dtype="<U4")

    open_index = np.where(structure == "(")[0]
    closed_index = np.where(structure == ")")[0]

    structure[open_index] = range(0, len(open_index))
    structure[closed_index] = range(len(open_index)-1, -1, -1)
    structure[structure == "."] = -1
    structure = structure.astype(int)

    pair_structure = np.array([-1]*len(structure))
    for i in range(len(open_index)):
        start, end = np.where(structure == i)[0]
        pair_structure[start] = end
        pair_structure[end] = start    
        
    return pair_structure

In [ ]:
def preprocess_inputs(df, cols=['sequence', 'structure', 'predicted_loop_type']):
    return np.transpose(
        np.array(
            df[cols]
            .applymap(lambda seq: [token2int[x] for x in seq])
            .values
            .tolist()
        ),
        (0, 2, 1)
    )

train_inputs_all = preprocess_inputs(train)
train_labels_all = np.array(train[target_cols].values.tolist()).transpose((0, 2, 1))

In [ ]:
# Building model (with my upgrade)

def MCRMSE(y_true, y_pred):
    colwise_mse = tf.reduce_mean(tf.square(y_true - y_pred), axis=1)
    return tf.reduce_mean(tf.sqrt(colwise_mse), axis=1)

def gru_layer(hidden_dim, dropout):
    return tf.keras.layers.Bidirectional(
                                tf.keras.layers.GRU(hidden_dim,
                                dropout=dropout,
                                return_sequences=True,
                                kernel_initializer = 'orthogonal'))

def lstm_layer(hidden_dim, dropout):
    return tf.keras.layers.Bidirectional(
                                tf.keras.layers.LSTM(hidden_dim,
                                dropout=dropout,
                                return_sequences=True,
                                kernel_initializer = 'orthogonal'))

def build_model(model_type=1, seq_len=107, pred_len=68, embed_dim=100, 
                dropout=dropout_model, hidden_dim_first = hidden_dim_first, 
                hidden_dim_second = hidden_dim_second, hidden_dim_third = hidden_dim_third):
    
    inputs = tf.keras.layers.Input(shape=(seq_len, 3))

    embed = tf.keras.layers.Embedding(input_dim=len(token2int), output_dim=embed_dim)(inputs)
    reshaped = tf.reshape(
        embed, shape=(-1, embed.shape[1],  embed.shape[2] * embed.shape[3]))
    
    reshaped = tf.keras.layers.SpatialDropout1D(.2)(reshaped)
    
    if model_type == 0:
        hidden = gru_layer(hidden_dim_first, dropout)(reshaped)
        hidden = gru_layer(hidden_dim_second, dropout)(hidden)
        hidden = gru_layer(hidden_dim_third, dropout)(hidden)
        
    elif model_type == 1:
        hidden = lstm_layer(hidden_dim_first, dropout)(reshaped)
        hidden = lstm_layer(hidden_dim_second, dropout)(hidden)
        hidden = lstm_layer(hidden_dim_third, dropout)(hidden)
        
    elif model_type == 2:
        hidden = gru_layer(hidden_dim_first, dropout)(reshaped)
        hidden = lstm_layer(hidden_dim_second, dropout)(hidden)
        hidden = lstm_layer(hidden_dim_third, dropout)(hidden)
        
    elif model_type == 3:
        hidden = lstm_layer(hidden_dim_first, dropout)(reshaped)
        hidden = gru_layer(hidden_dim_second, dropout)(hidden)
        hidden = gru_layer(hidden_dim_third, dropout)(hidden)

    elif model_type == 4:
        hidden = lstm_layer(hidden_dim_first, dropout)(reshaped)
        hidden = gru_layer(hidden_dim_second, dropout)(hidden)
        hidden = lstm_layer(hidden_dim_third, dropout)(hidden)
    
    truncated = hidden[:, :pred_len]

    out = tf.keras.layers.Dense(5, activation='linear')(truncated)

    model = tf.keras.Model(inputs=inputs, outputs=out)

    adam = tf.optimizers.Adam()
    model.compile(optimizer=adam, loss=MCRMSE)
    
    return model

In [ ]:
# Tunning model (with my upgrade)

def train_and_predict(n_folds=5, model_name="model", model_type=0, epochs=90, debug=False,
                      dropout_model=dropout_model, hidden_dim_first = hidden_dim_first, 
                      hidden_dim_second = hidden_dim_second, hidden_dim_third = hidden_dim_third,
                      seed=seed):

    print("Model:", model_name)

    ensemble_preds = pd.DataFrame(index=sample_sub.index, columns=target_cols).fillna(0) # test dataframe with 0 values
    kf = KFold(n_folds, shuffle=True, random_state=seed)
    skf = StratifiedKFold(n_folds, shuffle=True, random_state=seed)
    val_losses = []
    historys = []

    for i, (train_index, val_index) in enumerate(skf.split(train_inputs_all, train['SN_filter'])):
        print("Fold:", str(i+1))

        model_train = build_model(model_type=model_type, 
                                  dropout=dropout_model, 
                                  hidden_dim_first = hidden_dim_first, 
                                  hidden_dim_second = hidden_dim_second, 
                                  hidden_dim_third = hidden_dim_third)
        model_short = build_model(model_type=model_type, seq_len=107, pred_len=107,
                                  dropout=dropout_model, 
                                  hidden_dim_first = hidden_dim_first, 
                                  hidden_dim_second = hidden_dim_second, 
                                  hidden_dim_third = hidden_dim_third)
        model_long = build_model(model_type=model_type, seq_len=130, pred_len=130,
                                 dropout=dropout_model, 
                                 hidden_dim_first = hidden_dim_first, 
                                 hidden_dim_second = hidden_dim_second, 
                                 hidden_dim_third = hidden_dim_third)

        train_inputs, train_labels = train_inputs_all[train_index], train_labels_all[train_index]
        val_inputs, val_labels = train_inputs_all[val_index], train_labels_all[val_index]

        checkpoint = tf.keras.callbacks.ModelCheckpoint(f'{model_name}.h5')

        history = model_train.fit(
            train_inputs , train_labels, 
            validation_data=(val_inputs,val_labels),
            batch_size=64,
            epochs=epochs, # changed 70
            callbacks=[tf.keras.callbacks.ReduceLROnPlateau(), checkpoint],
            verbose=2 if debug else 0
        )

        print(f"{model_name} Min training loss={min(history.history['loss'])}, min validation loss={min(history.history['val_loss'])}")

        val_losses.append(min(history.history['val_loss']))
        historys.append(history)

        model_short.load_weights(f'{model_name}.h5')
        model_long.load_weights(f'{model_name}.h5')

        public_preds = model_short.predict(public_inputs)
        private_preds = model_long.predict(private_inputs)

        preds_model = []
        for df, preds in [(public_df, public_preds), (private_df, private_preds)]:
            for i, uid in enumerate(df.id):
                single_pred = preds[i]

                single_df = pd.DataFrame(single_pred, columns=target_cols)
                single_df['id_seqpos'] = [f'{uid}_{x}' for x in range(single_df.shape[0])]

                preds_model.append(single_df)

        preds_model_df = pd.concat(preds_model)
        ensemble_preds[target_cols] += preds_model_df[target_cols].values / n_folds

        if debug:
            print("Intermediate ensemble result")
            print(ensemble_preds[target_cols].head())

    ensemble_preds["id_seqpos"] = preds_model_df["id_seqpos"].values
    ensemble_preds = pd.merge(sample_sub["id_seqpos"], ensemble_preds, on="id_seqpos", how="left")

    print("Mean Validation loss:", str(np.mean(val_losses)))

    if debug:
        fig, ax = plt.subplots(1, 1, figsize = (10, 5))
        for i, history in enumerate(historys):
            ax.plot(history.history['loss'])
            ax.plot(history.history['val_loss'])
            ax.set_title('model_'+str(i+1))
            ax.set_ylabel('Loss')
            ax.set_xlabel('Epoch')
        plt.show()

    return ensemble_preds


public_df = test.query("seq_length == 107").copy()
private_df = test.query("seq_length == 130").copy()
public_inputs = preprocess_inputs(public_df)
private_inputs = preprocess_inputs(private_df)

ensembles = []

for i in range(5):
    model_name = "model_"+str(i+1)

    ensemble = train_and_predict(n_folds=5, model_name=model_name, model_type=i, epochs=100,
                                 dropout_model=dropout_model, hidden_dim_first = hidden_dim_first, 
                                 hidden_dim_second = hidden_dim_second, hidden_dim_third = hidden_dim_third,
                                 seed=seed)
    ensembles.append(ensemble)

## 4. Ensembling the solutions and submission <a class="anchor" id="4"></a>

[Back to Table of Contents](#0.1)

In [ ]:
# Ensembling the solutions
ensemble_final = ensembles[0].copy()
ensemble_final[target_cols] = 0

for ensemble in ensembles:
    ensemble_final[target_cols] += ensemble[target_cols].values / len(ensembles)

ensemble_final

In [ ]:
# Submission
ensemble_final.to_csv('ensemble_final.csv', index=False)

I hope you find my upgrade of the original notebooks useful and enjoyable.

Your comments and feedback are most welcome.

[Go to Top](#0)